#Assignment04. Origin-Destination Analysis with NetworkX

##Install and Import Packages

In [ ]:
# Install packages

!pip install fiona
!pip install geopandas
!pip install folium
!pip install sodapy -q
!pip install adjustText
!pip install plotly --upgrade
!pip install contextily
!pip install mapclassify
!pip install chart-studio
!sudo apt install libspatialindex-dev
!pip install rtree
!pip install pygeos

     |████████████████████████████████| 15.4 MB 12.2 MB/s 
     |████████████████████████████████| 1.0 MB 16.4 MB/s 
     |████████████████████████████████| 6.3 MB 36.0 MB/s 
  Created wheel for adjustText: filename=adjustText-0.7.3-py3-none-any.whl size=7094 sha256=f4f4ec54d38cc45ed6d3175064888b717824bfa29161868d0c03dbe605e9e521
  Stored in directory: /root/.cache/pip/wheels/2f/98/32/afbf902d8f040fadfdf0a44357e4ab750afe165d873bf5893d
Successfully built adjustText
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 19.3 MB 1.2 MB/s 
     |████████████████████████████████| 64 kB 124 kB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
The following NE

In [ ]:
!pip install decorator
!pip install pandapower

     |████████████████████████████████| 5.8 MB 6.7 MB/s 
     |████████████████████████████████| 149 kB 15.3 MB/s 
     |████████████████████████████████| 3.6 MB 43.9 MB/s 
  Created wheel for pandapower: filename=pandapower-2.7.0-py3-none-any.whl size=5777359 sha256=a115a0fd0cd1cbc7e7b63e2615fb98adb4c221e2d710d12181fcc655f1be8e6f
  Stored in directory: /root/.cache/pip/wheels/ff/e6/b6/c9218d5a45a02f4bdb220272f2bac4f59c5b73ed08a08125c3
Successfully built pandapower


In [ ]:
# Import packages
# To ignore unimporant system warnings
import warnings
warnings.filterwarnings("ignore")

# We will use Pandas, Numpy, and Matplotlib which is a package for visualization with Python
import pandas as pd
import numpy as np

# Load a required package 
# This is a library for accessing and parsing data through URLs
from urllib.parse import urlencode
import urllib.request, json 
from bs4 import BeautifulSoup # for web scraping
import fiona
import plotly.express as px
import geopandas as gpd
import adjustText as aT
import seaborn as sns # visualization styling package
import plotly.graph_objects as go
import urllib, json
import chart_studio.plotly as py
import chart_studio.tools as tls
import os
import plotly
from sodapy import Socrata
from IPython.display import Image, display
from shapely.geometry import shape
from shapely.geometry import Point 
from shapely.geometry import Polygon
import time
from shapely.geometry import shape
import contextily as ctx
from google.colab import files
from sklearn.preprocessing import MinMaxScaler

import networkx as nx #library supporting networks

%matplotlib inline 
import networkx as nx #library supporting networks
import matplotlib.pyplot as plt
import seaborn as sns # visualization styling package
import folium
from folium.plugins import HeatMap

#Chart Studio API
import chart_studio
username='bkirthi'
api_key='PHHYaOZKZDA7BF3nUUtO'
chart_studio.tools.set_credentials_file(username=username,
                                        api_key=api_key)

In [ ]:
# import more packages
import plotly.io as pio
pio.renderers.default = "colab"
access_token = 'pk.eyJ1IjoiYmtpcnRoaSIsImEiOiJja3VpdzFiMnoycnYyMzBtYXM1aW1uemd0In0.knHw7eYrN8DvrrBHZugW8Q'
px.set_mapbox_access_token(access_token) # for interactive plotly viz

import plotly.graph_objects as go
import chart_studio.plotly as py
import chart_studio.tools as tls

##Import Citibike Rideshare Data
Downloadable from https://www.citibikenyc.com/system-data. Click “Downloadable files of Citi Bike trip data” or “Download Citi Bike trip history data”.

Here, download the data for August 2021.


In [ ]:
# clone github
!git clone https://github.com/kirthi-b/CitiBike-NetworkX

Cloning into 'CitiBike-NetworkX'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 32 (delta 10), reused 24 (delta 5), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [ ]:
feb = pd.read_csv("/content/CitiBike-NetworkX/Data/202102.csv")
may = pd.read_csv("/content/CitiBike-NetworkX/Data/202105.csv")
aug = pd.read_csv("/content/CitiBike-NetworkX/Data/202108.csv")
nov = pd.read_csv("/content/CitiBike-NetworkX/Data/202111.csv")

In [ ]:
aug.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5CABAE380FF3C7E0,classic_bike,2021-08-06 07:02:02,2021-08-06 07:28:56,Dey St,JC065,Stevens - River Ter & 6 St,HB602,40.737711,-74.066921,40.743133,-74.026989,casual
1,0C820F4E9256B5BB,classic_bike,2021-08-15 19:48:51,2021-08-15 20:04:45,Grand St & 2 St,HB405,Jersey & 3rd,JC074,40.739130,-74.036180,40.723332,-74.045953,member
2,F33B1B767F55A2BC,classic_bike,2021-08-26 17:14:58,2021-08-26 17:20:13,Grand St & 2 St,HB405,7 St & Monroe St,HB304,40.739130,-74.036180,40.746413,-74.037977,casual
3,EF7A1541C2302EEF,classic_bike,2021-08-12 12:58:22,2021-08-12 13:04:07,Dey St,JC065,Sip Ave,JC056,40.737711,-74.066921,40.730897,-74.063913,member
4,74DEC24E4C66BA01,classic_bike,2021-08-04 20:59:45,2021-08-04 21:10:39,Glenwood Ave,JC094,Pershing Field,JC024,40.727551,-74.071061,40.742677,-74.051789,member


In [ ]:
print(feb.shape[0])
print(may.shape[0])
print(aug.shape[0])
print(nov.shape[0])

(4881, 13)
(45043, 13)
(93800, 13)
(60061, 13)


In [ ]:
#get list of all routes
tot = pd.concat([feb,may,aug,nov])
tot_count = tot.groupby(['start_station_name']).size().to_frame('count').reset_index()
tot_df = pd.DataFrame(tot_count)
tot.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,121DD7DD23CB1335,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,Hoboken Ave at Monmouth St,JC105,Christ Hospital,JC034,40.735208,-74.046964,40.734786,-74.050444,member
1,FD73FB85F008349D,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,Newport Pkwy,JC008,Marin Light Rail,JC013,40.728744,-74.032108,40.714584,-74.042817,member
2,39F9E6663CB5FDF6,docked_bike,2021-02-26 23:16:04,2021-02-26 23:22:25,Journal Square,JC103,Baldwin at Montgomery,JC020,40.733670,-74.062500,40.723659,-74.064194,member
3,A64745CB0792EC6F,docked_bike,2021-02-24 16:51:50,2021-02-24 17:16:09,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964,casual
4,75CC76EB9543764A,docked_bike,2021-02-24 20:44:16,2021-02-24 20:44:46,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964,member


In [ ]:
tot['started_at'] = pd.to_datetime(tot['started_at'])
tot['date'] = tot['started_at'].dt.date
tot['date'] = tot['date'].astype(str)
tot.shape[0]

203785

In [ ]:
tot_sub = tot[['end_station_name','end_lat','end_lng','date']]
tot_sub = tot_sub.drop_duplicates()
tot_sub.shape[0]

9679

In [ ]:
tot_df.head()

,start_station_name,count
0,11 St & Washington St,4882
1,12 St & Sinatra Dr N,1709
2,14 St Ferry - 14 St & Shipyard Ln,3605
3,2 St HBLR - 2 St & Marshall St,228
4,4 St & Grand St,1872


In [ ]:
import plotly.io as pio
pio.renderers.default = "colab"
access_token = 'pk.eyJ1IjoiYmtpcnRoaSIsImEiOiJja3VpdzFiMnoycnYyMzBtYXM1aW1uemd0In0.knHw7eYrN8DvrrBHZugW8Q'
px.set_mapbox_access_token(access_token) # for interactive plotly viz

import plotly.graph_objects as go
import chart_studio.plotly as py
import chart_studio.tools as tls

In [ ]:
fig = px.scatter_mapbox(tot_sub,
                    lat='end_lat',
                    lon='end_lng',
                        color_discrete_sequence=px.colors.qualitative.Pastel,
                    animation_frame='date',
                       mapbox_style='dark')

fig.update_layout(autosize=False,
                 margin = dict(l = 0, r = 0, t = 0, b = 0),
                 paper_bgcolor ='rgba(0,0,0,0)',
                 font_color= '#aaa'
                 )

fig.update_layout(mapbox_zoom=10, 
                  mapbox_center = {"lat": 40.776789, "lon": -73.970212})
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [ ]:
py.plot(fig, filename="citibike", auto_open = True)

'https://plotly.com/~bkirthi/44/'

In [ ]:
fig = px.bar(tot_sub, x='date', y='counts',
             hover_data=['ym', 'counts'], color='borough',
             labels={'counts':'Noise complaint volume','ym':'Month'}, height=400)
fig.show()